In [4]:
from lnas import LnasFormat
import pathlib
import numpy as np

NORMAL_OFFSET = 0.125

block_lbl = "G100"

meshes_path = pathlib.Path(
    "/home/ubuntu/Documentos/Repositories/insight/Docker/local/volume/divided_cp"
)

mesh_path = meshes_path / f"{block_lbl}.lnas"
neg_mesh_path = mesh_path.parent / f"{block_lbl}_negative.lnas"
output_path = mesh_path.parent / f"{block_lbl}.merged.lnas"

mesh = LnasFormat.from_file(mesh_path)
neg_mesh = LnasFormat.from_file(neg_mesh_path)

geometry_negative = neg_mesh.geometry
geometry_negative.triangles = np.flip(geometry_negative.triangles, 1)
geometry_negative._full_update()
geometry_negative.vertices = (
    geometry_negative.vertices + geometry_negative.vertices_normals * NORMAL_OFFSET
)

mesh.join(lnas_fmts=[neg_mesh], surfaces_suffixes=["_inside"])
mesh.to_file(output_path)

Join grouped hist series

In [5]:
import pandas as pd
import pathlib

data_path = pathlib.Path(
    "/home/ubuntu/Documentos/Repositories/insight/Docker/local/volume/divided_cp"
)
block_lbl = "G100"

(data_path / f"bodies.{block_lbl}.merged.data.h5").unlink()

with pd.HDFStore(data_path / f"bodies.{block_lbl}_hs.data.h5", mode="r") as body_store:
    with pd.HDFStore(data_path / f"bodies.{block_lbl}_neg_hs.data.h5", mode="r") as neg_body_store:
        body_groups = body_store.keys()
        neg_body_groups = neg_body_store.keys()

        if body_groups != neg_body_groups:
            raise Exception("Keys for body and negative don't match!")

        for store_group in body_groups:
            body_hist_series = body_store.get(store_group)
            negative_body_hist_series = neg_body_store.get(store_group)

            last_index = body_hist_series.point_idx.max()
            negative_body_hist_series.point_idx += last_index + 1

            new_df = pd.concat([body_hist_series, negative_body_hist_series]).sort_values(
                by=["time_step", "point_idx"]
            )
            new_df.to_hdf(
                path_or_buf=data_path / f"bodies.{block_lbl}.merged.data.h5",
                key=store_group,
                mode="a",
            )

['/step0005000', '/step0006000', '/step0007000', '/step0008000', '/step0009000', '/step0010000', '/step0011000', '/step0012000', '/step0013000', '/step0014000', '/step0015000', '/step0016000', '/step0017000', '/step0018000', '/step0019000', '/step0020000']


Join complete hist series

In [ ]:
import pandas as pd
import pathlib

data_path = pathlib.Path(
    "/home/ubuntu/Documentos/Repositories/insight/Docker/local/volume/divided_cp"
)
block_lbl = "G100"

body_hist_series = pd.read_hdf(data_path / f"bodies.{block_lbl}_hs.data.h5")
negative_body_hist_series = pd.read_hdf(data_path / f"bodies.{block_lbl}_neg_hs.data.h5")

last_index = body_hist_series.point_idx.max()
negative_body_hist_series.point_idx += last_index + 1

new_df = pd.concat([body_hist_series, negative_body_hist_series]).sort_values(
    by=["time_step", "point_idx"]
)
new_df.to_hdf(
    path_or_buf=data_path.parent / f"bodies.{block_lbl}.merged.data.h5", key="df", mode="w"
)